In [1]:
import pandas as pd
import numpy as np
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble  import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

2021-12-10 13:49:23.249121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 13:49:23.249137: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Definindo o caminho dos dados
data_company_path = "dados/Company.csv"
data_tweet_path = "dados/Tweet.csv"
data_company_tweet_path = "dados/Company_Tweet.csv"

In [3]:
# Ler e colocar em um dataframe
data_company = pd.read_csv(data_company_path)
data_tweet = pd.read_csv(data_tweet_path)
data_company_tweet = pd.read_csv(data_company_tweet_path)

In [4]:
print(data_company_tweet.head())
print(data_company_tweet.shape)

             tweet_id ticker_symbol
0  550803612197457920          AAPL
1  550803610825928706          AAPL
2  550803225113157632          AAPL
3  550802957370159104          AAPL
4  550802855129382912          AAPL
(4336445, 2)


In [5]:
print(data_company)
print(data_company.shape)

  ticker_symbol company_name
0          AAPL        apple
1          GOOG   Google Inc
2         GOOGL   Google Inc
3          AMZN   Amazon.com
4          TSLA    Tesla Inc
5          MSFT    Microsoft
(6, 2)


In [6]:
print(data_tweet.head())
print(data_tweet.shape)

             tweet_id           writer   post_date  \
0  550441509175443456  VisualStockRSRC  1420070457   
1  550441672312512512      KeralaGuy77  1420070496   
2  550441732014223360      DozenStocks  1420070510   
3  550442977802207232     ShowDreamCar  1420070807   
4  550443807834402816     i_Know_First  1420071005   

                                                body  comment_num  \
0  lx21 made $10,008  on $AAPL -Check it out! htt...            0   
1  Insanity of today weirdo massive selling. $aap...            0   
2  S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...            0   
3  $GM $TSLA: Volkswagen Pushes 2014 Record Recal...            0   
4  Swing Trading: Up To 8.91% Return In 14 Days h...            0   

   retweet_num  like_num  
0            0         1  
1            0         0  
2            0         0  
3            0         1  
4            0         1  
(3717964, 7)


In [7]:
dados_merge1 = pd.merge(data_company_tweet,data_company, on='ticker_symbol')
dados_merge1 = dados_merge1.drop(columns=['ticker_symbol'])
dados_merge1.head()

,tweet_id,company_name
0,550803612197457920,apple
1,550803610825928706,apple
2,550803225113157632,apple
3,550802957370159104,apple
4,550802855129382912,apple


In [8]:
dados_tweet_company = pd.merge(dados_merge1,data_tweet,on='tweet_id')
dados_tweet_company = dados_tweet_company.drop(columns=['tweet_id'])
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [11]:
dados_tweet_company = dados_tweet_company.iloc[0:10000]
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [12]:
dados_tweet_company.shape

(10000, 7)

## Ajustar os Dados

In [13]:
dados_tweet_company.post_date[0]

1420156789

In [14]:
ts_epoch = dados_tweet_company.post_date[0]
ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
ts

'2015-01-01 20:59:49'

In [15]:
def ajustar_tempo(row):
    ts_epoch = row['post_date']
    ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y-%m-%d %H:%M:%S')
    return ts  
new_date = dados_tweet_company.apply(ajustar_tempo,axis=1)
dados_tweet_company['Date'] = list(new_date)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48


In [16]:
dados_tweet_company.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  10000 non-null  object
 1   writer        9895 non-null   object
 2   post_date     10000 non-null  int64 
 3   body          10000 non-null  object
 4   comment_num   10000 non-null  int64 
 5   retweet_num   10000 non-null  int64 
 6   like_num      10000 non-null  int64 
 7   Date          10000 non-null  object
dtypes: int64(4), object(4)
memory usage: 961.2+ KB


In [17]:
sentenca = dados_tweet_company.body[0]
print(sentenca)

sentenca_2 = dados_tweet_company.body[100]
print(sentenca_2)

#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN $T $IGN $SPY $FB $UGAZ #sentiquant 20150101 19:00:08:298
Our Penny Stock Picks Gained Over 968% In the Past 5 Weeks! Get our next pick early: http://tinyurl.com/p46mnk2 $TRX $AAPL $GOOG


In [18]:
def replace_all(texto):
    palavras_especiais = [word for word in texto.split(' ') if word.startswith(('#','$','@'))]
    #print(palavras_especiais)
    sentenca = ' '.join(x for x in texto.split(' ') if x not in palavras_especiais)
    return sentenca

new_strings = dados_tweet_company.body.apply(replace_all)
dados_tweet_company['Processed_Tweet'] = list(new_strings)
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526


In [19]:
'https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664'

'https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664'

In [20]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(dados_tweet_company.body[0])

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [21]:
dados_tweet_company.shape

(10000, 9)

In [22]:
dados_tweet_company['vader_scores']  = None
dados_tweet_company['vader_scores'] = dados_tweet_company.Processed_Tweet.apply(lambda tweet : sid.polarity_scores(tweet))
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [23]:
# separar os valores do compound - vamos uas-lo para determinar os lbels dos tweets pos/beg
dados_tweet_company['compound']  = dados_tweet_company['vader_scores'].apply(lambda score_dict: score_dict['compound'])
dados_tweet_company.head()


,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0


In [24]:
# definindo os labels usando o compund value como referenecia
def labels(value):
    if value == 0:
        return 0
    elif value > 0:
        return 1
    elif value < 0:
        return -1
    
dados_tweet_company['label']  = dados_tweet_company['compound'].apply(lambda value: labels(value))
dados_tweet_company.head()

,company_name,writer,post_date,body,comment_num,retweet_num,like_num,Date,Processed_Tweet,vader_scores,compound,label
0,apple,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
1,Amazon.com,SentiQuant,1420156789,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,2015-01-01 20:59:49,20150101 19:00:08:298,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
2,apple,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
3,Google Inc,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0
4,Amazon.com,SentiQuant,1420156788,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,2015-01-01 20:59:48,20150101 19:00:04:526,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0


In [25]:
dados_tweet_company.label.value_counts()

 1    4711
 0    4102
-1    1187
Name: label, dtype: int64

In [26]:
# separando os valores para a machine learning
dados = dados_tweet_company[['Processed_Tweet','label']]
dados.head()

,Processed_Tweet,label
0,20150101 19:00:08:298,0
1,20150101 19:00:08:298,0
2,20150101 19:00:04:526,0
3,20150101 19:00:04:526,0
4,20150101 19:00:04:526,0


In [27]:
datasetVariables_train, datasetVariables_test, datasetTarget_train, datasetTarget_test = train_test_split(dados['Processed_Tweet'], dados['label'], test_size=0.33, random_state=42)

In [28]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(datasetVariables_train)

word_index = tokenizer.word_index
print(len(word_index))

8096


In [30]:
print(word_index)

{'<OOV>': 1, 'http': 2, 'in': 3, 'for': 4, 'the': 5, 'apple': 6, 'me': 7, 'com': 8, 'to': 9, 'free': 10, 'first': 11, '100': 12, 'adyou': 13, '000': 14, 'account': 15, 'members': 16, 'balance': 17, '5€': 18, 'on': 19, 'a': 20, 'is': 21, 'of': 22, 'and': 23, '2015': 24, 'tqm': 25, 'up': 26, 'ly': 27, 'tt': 28, 'ift': 29, 'you': 30, 'it': 31, 'stock': 32, 'co': 33, 'my': 34, 'at': 35, 'stks': 36, 'with': 37, 'over': 38, 'watch': 39, 'i': 40, 'get': 41, '1vwqqqa': 42, 'bit': 43, 'this': 44, 'options': 45, 'will': 46, '1': 47, 'goo': 48, 'gl': 49, 'here': 50, 'our': 51, 'iphone': 52, 'be': 53, 'when': 54, 'if': 55, 'source': 56, 'seekingalpha': 57, 'all': 58, 'article': 59, 'win': 60, 'are': 61, 'w': 62, 'now': 63, 'link': 64, 'market': 65, 'us': 66, 'from': 67, 'aapl': 68, 'binary': 69, 'competition': 70, 'out': 71, 'new': 72, 'that': 73, '200': 74, 'sign': 75, 'available': 76, '6': 77, 'news': 78, 'worldwide': 79, 'countries': 80, 'feed': 81, '…': 82, 'r0z': 83, 'bonus': 84, 'stocks': 85

In [31]:
# Transformar as sentencas dos dados de treinamento e teste para sequencias numericas. ex [[1,2,6],[65,952,15,65],[1,2]]
train_data_sentences = tokenizer.texts_to_sequences(datasetVariables_train)
test_data_sentences = tokenizer.texts_to_sequences(datasetVariables_test)

# normalizar o tamnho das sequencias usando padding. ex [[0,1,2,6],[65,952,15,65],[0,0,1,2]]
train_padded = pad_sequences(train_data_sentences, maxlen=300)
test_padded = pad_sequences(test_data_sentences, maxlen=300)

print("Conjunto de treinamento: ","\n",train_padded[0:19])
print("Conjunto de teste: ","\n",test_padded[0:19])

Conjunto de treinamento:  
 [[   0    0    0 ...   13    7  109]
 [   0    0    0 ...  766  357  200]
 [   0    0    0 ...   98  289 4000]
 ...
 [   0    0    0 ...   56   81  131]
 [   0    0    0 ...  727  159  377]
 [   0    0    0 ...    9  151 4010]]
Conjunto de teste:  
 [[   0    0    0 ...   13    7   25]
 [   0    0    0 ...  484  485  486]
 [   0    0    0 ...   13    7  148]
 ...
 [   0    0    0 ...  150   27 5162]
 [   0    0    0 ...    2  172    8]
 [   0    0    0 ...   80   79   84]]


In [38]:
from tensorflow.keras.utils import to_categorical

datasetTarget_train = to_categorical(np.array(datasetTarget_train))
datasetTarget_test = to_categorical(np.array(datasetTarget_test))

In [33]:
from math import sqrt

In [40]:
# first ML model
model_v1 = RandomForestClassifier(random_state=50,criterion='gini')
model_v1.fit(train_padded,datasetTarget_train)
# model_v1 acuracy
model_v1Score = model_v1.score(test_padded,datasetTarget_test)
model_v1Score

0.8196969696969697

In [41]:
# save model
import pickle
filename = 'model_randomForest_80.sav'
pickle.dump(model_v1,open(filename,'wb'))